A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below

In [5]:
#Reading the file
rental = pd.read_csv('rental_info.csv')
rental.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


In [6]:
rental['rental_length_days'] = (pd.to_datetime(rental['return_date']) - pd.to_datetime(rental['rental_date'])).dt.days
rental['rental_length_days'].head()

0    3
1    2
2    7
3    2
4    4
Name: rental_length_days, dtype: int64

In [7]:
rental['deleted_scenes'] = np.where(rental["special_features"].str.contains("Deleted Scenes"),1,0)
rental['deleted_scenes'].head()
rental['behind_the_scenes'] = np.where(rental['special_features'].str.contains("Behind the Scenes"), 1, 0)
rental['behind_the_scenes'].head()

0    1
1    1
2    1
3    1
4    1
Name: behind_the_scenes, dtype: int64

In [8]:
#Separating features and target values
X = rental.drop(['rental_length_days','special_features','rental_date', 'return_date'], axis = 1)
y = rental['rental_length_days'].copy()

In [9]:
#train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state =42)

In [10]:
#Performing Feature Selection
from sklearn.linear_model import Lasso

lasso = Lasso(alpha = 0.1, random_state = 9)
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

In [11]:
#Linear Regression model
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()
linear_reg.fit(X_lasso_train, y_train)
linear_predictions = linear_reg.predict(X_lasso_test)
linear_mse = mean_squared_error(y_test, linear_predictions) 
linear_mse

4.856095723959099

In [12]:
#Random forest model
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist = {
    'n_estimators' :  np.arange(1, 101, 1),
    'max_depth' : np.arange(1, 11, 1)
}

rf = RandomForestRegressor()
rand_search = RandomizedSearchCV(rf, param_distributions=param_dist, cv = 6, random_state = 9)
rand_search.fit(X_train, y_train)
hyper_params = rand_search.best_params_
rf = RandomForestRegressor(n_estimators = hyper_params['n_estimators'],
                          max_depth = hyper_params['max_depth'],
                          random_state= 9)
rf.fit(X_train, y_train)
rf_predictions = rf.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_mse

2.136758362052645

In [13]:
best_model = rf
best_mse = rf_mse